# Air Quality Data

Resource used: https://api-ninjas.com/api/airquality

In [1]:
import requests
import json
import pandas as pd

## API Request to access Air Quality Information

### Information gathered for 100 cities around the DMV

In [12]:
cities = [
    'Washington', 'Arlington', 'Alexandria', 'Fairfax', 'Rockville', 
    'Bethesda', 'Silver Spring', 'Reston', 'Frederick', 'Gaithersburg',
    'Annapolis', 'Bowie', 'College Park', 'Greenbelt', 'Laurel',
    'Cumberland', 'Hagerstown', 'Salisbury', 'Westminster', 'Hyattsville',
    'Takoma Park', 'Chevy Chase', 'Kensington', 'Poolesville', 'Bladensburg',
    'Mount Rainier', 'Riverdale Park', 'Berwyn Heights', 'New Carrollton', 'Glenarden',
    'Brentwood', 'Capitol Heights', 'Colmar Manor', 'Cottage City', 'Cheverly',
    'Edmonston', 'Fairmount Heights', 'Forest Heights', 'Landover Hills', 'Morningside',
    'North Brentwood', 'Seat Pleasant', 'University Park', 'Upper Marlboro', 'Vienna',
    'Falls Church', 'Manassas', 'Leesburg', 'Herndon', 'Purcellville',
    'Warrenton', 'Culpeper', 'Winchester', 'Front Royal', 'Strasburg',
    'Woodstock', 'Luray', 'Berryville', 'Middleburg', 'Dumfries',
    'Occoquan', 'Haymarket', 'Clifton', 'Quantico', 'Hamilton',
    'Hillsboro', 'Lovettsville', 'Round Hill', 'Ashburn', 'Chantilly',
    'Sterling', 'Great Falls', 'McLean', 'Tysons', 'Burke',
    'Springfield', 'Lorton', 'Mount Vernon', 'Annandale', 'Oakton',
    'Falls Church', 'Centreville', 'Manassas Park', 'Gainesville', 'Bristow',
    'Nokesville', 'Dale City', 'Woodbridge', 'Stafford', 'Fredericksburg', 
    'Potomac', 'Ellicott City', 'Germantown', 'Waldorf', 'Lanham',
    'Beltsville', 'Crofton', 'Oxon Hill', 'Sykesville', 'Olney', 'Georgetown', 'Dupont Circle'
]

In [13]:

api_url = 'https://api.api-ninjas.com/v1/airquality?city={}'
api_key = 'rtsVUhXvel9N968aGEoMeg==zciz73lsmuFNEOQM'

# Dictionary to store the results
results = {}

# Loop through each city and make an API request
for city in cities:
    response = requests.get(api_url.format(city), headers={'X-Api-Key': api_key})
    
    if response.status_code == requests.codes.ok:
        results[city] = response.json()
    else:
        print("Error in city:", city, "-", response.status_code, response.text)

# Save the results in a JSON file
with open('air_quality_data.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

print("Data saved in air_quality_data.json")

Data saved in air_quality_data.json


## Read Data from JSON File and Save as a Dataframe

In [14]:
# Load the JSON data from the file
file_path = 'air_quality_data.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Convert the JSON data to a DataFrame
df_list = []
for city, metrics in data.items():
    row = {
        'city': city,
        'CO': metrics.get('CO'),
        'NO2': metrics.get('NO2'),
        'O3': metrics.get('O3'),
        'SO2': metrics.get('SO2'),
        'PM2.5': metrics.get('PM2.5'),
        'PM10': metrics.get('PM10'),
        'overall_aqi': metrics.get('overall_aqi')
    }
    df_list.append(row)

df = pd.DataFrame(df_list)

df.head()


,city,CO,NO2,O3,SO2,PM2.5,PM10,overall_aqi
0,Washington,"{'concentration': 293.73, 'aqi': 3}","{'concentration': 13.02, 'aqi': 16}","{'concentration': 62.94, 'aqi': 62}","{'concentration': 4.47, 'aqi': 6}","{'concentration': 9.21, 'aqi': 29}","{'concentration': 11.43, 'aqi': 10}",62
1,Arlington,"{'concentration': 527.38, 'aqi': 5}","{'concentration': 67.86, 'aqi': 84}","{'concentration': 7.06, 'aqi': 5}","{'concentration': 1.74, 'aqi': 2}","{'concentration': 15.09, 'aqi': 48}","{'concentration': 20.53, 'aqi': 19}",84
2,Alexandria,"{'concentration': 211.95, 'aqi': 2}","{'concentration': 0.73, 'aqi': 0}","{'concentration': 91.55, 'aqi': 141}","{'concentration': 0.61, 'aqi': 0}","{'concentration': 22.61, 'aqi': 64}","{'concentration': 55.66, 'aqi': 50}",141
3,Fairfax,"{'concentration': 273.71, 'aqi': 3}","{'concentration': 8.74, 'aqi': 10}","{'concentration': 69.38, 'aqi': 82}","{'concentration': 4.95, 'aqi': 7}","{'concentration': 8.53, 'aqi': 27}","{'concentration': 10.41, 'aqi': 9}",82
4,Rockville,"{'concentration': 287.06, 'aqi': 3}","{'concentration': 10.8, 'aqi': 13}","{'concentration': 62.94, 'aqi': 62}","{'concentration': 4.65, 'aqi': 6}","{'concentration': 8.77, 'aqi': 28}","{'concentration': 10.77, 'aqi': 9}",62


## Clean the DF

In [15]:
# Separate the concentration and AQI values into distinct columns
for pollutant in ['CO', 'NO2', 'O3', 'SO2', 'PM2.5', 'PM10']:
    df[f'{pollutant}_concentration'] = df[pollutant].apply(lambda x: x.get('concentration'))
    df[f'{pollutant}_aqi'] = df[pollutant].apply(lambda x: x.get('aqi'))
    df.drop(columns=[pollutant], inplace=True)

# Reorder the columns for better readability
columns_order = ['city', 'CO_concentration', 'CO_aqi', 'NO2_concentration', 'NO2_aqi', 
                 'O3_concentration', 'O3_aqi', 'SO2_concentration', 'SO2_aqi', 
                 'PM2.5_concentration', 'PM2.5_aqi', 'PM10_concentration', 'PM10_aqi', 'overall_aqi']
df = df[columns_order]

df

,city,CO_concentration,CO_aqi,NO2_concentration,NO2_aqi,O3_concentration,O3_aqi,SO2_concentration,SO2_aqi,PM2.5_concentration,PM2.5_aqi,PM10_concentration,PM10_aqi,overall_aqi
0,Washington,293.73,3,13.02,16,62.94,62,4.47,6,9.21,29,11.43,10,62
1,Arlington,527.38,5,67.86,84,7.06,5,1.74,2,15.09,48,20.53,19,84
2,Alexandria,211.95,2,0.73,0,91.55,141,0.61,0,22.61,64,55.66,50,141
3,Fairfax,273.71,3,8.74,10,69.38,82,4.95,7,8.53,27,10.41,9,82
4,Rockville,287.06,3,10.80,13,62.94,62,4.65,6,8.77,28,10.77,9,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Oxon Hill,360.49,4,26.39,32,43.63,36,4.47,6,11.41,37,14.03,12,37
97,Sykesville,283.72,3,9.25,11,62.94,62,3.93,5,8.44,27,10.50,9,62
98,Olney,253.68,2,6.00,7,28.97,24,0.03,0,5.09,16,5.66,5,24
99,Georgetown,181.91,2,0.03,0,60.08,53,0.10,0,2.49,8,7.90,7,53


In [16]:
virginia_cities = {
    'Arlington', 'Alexandria', 'Fairfax', 'Reston', 'Vienna', 'Falls Church', 'Manassas', 'Leesburg', 'Herndon', 
    'Purcellville', 'Warrenton', 'Culpeper', 'Winchester', 'Front Royal', 'Strasburg', 'Woodstock', 'Luray', 
    'Berryville', 'Middleburg', 'Dumfries', 'Occoquan', 'Haymarket', 'Clifton', 'Quantico', 'Hamilton', 'Hillsboro', 
    'Lovettsville', 'Round Hill', 'Ashburn', 'Chantilly', 'Sterling', 'Great Falls', 'McLean', 'Tysons', 'Burke', 
    'Springfield', 'Lorton', 'Mount Vernon', 'Annandale', 'Oakton', 'Centreville', 'Manassas Park', 'Gainesville', 
    'Bristow', 'Nokesville', 'Dale City', 'Woodbridge', 'Stafford', 'Fredericksburg'
}

maryland_cities = {
    'Rockville', 'Bethesda', 'Silver Spring', 'Frederick', 'Gaithersburg', 'Annapolis', 'Bowie', 'College Park', 
    'Greenbelt', 'Laurel', 'Cumberland', 'Hagerstown', 'Salisbury', 'Westminster', 'Hyattsville', 'Takoma Park', 
    'Chevy Chase', 'Kensington', 'Poolesville', 'Bladensburg', 'Mount Rainier', 'Riverdale Park', 'Berwyn Heights', 
    'New Carrollton', 'Glenarden', 'Brentwood', 'Capitol Heights', 'Colmar Manor', 'Cottage City', 'Cheverly', 
    'Edmonston', 'Fairmount Heights', 'Forest Heights', 'Landover Hills', 'Morningside', 'North Brentwood', 
    'Seat Pleasant', 'University Park', 'Upper Marlboro', 'Potomac', 'Ellicott City', 'Germantown', 'Waldorf', 
    'Lanham', 'Beltsville', 'Crofton', 'Oxon Hill', 'Sykesville', 'Olney'
}

washington_dc = {'Washington', 'Georgetown', 'Dupont Circle'}

df1 = df.copy()

df1['area'] = df1['city'].apply(lambda city: 'Virginia' if city in virginia_cities 
                                        else 'Maryland' if city in maryland_cities 
                                        else 'Washington D.C.' if city in washington_dc
                                        else 'Unknown')

df1

,city,CO_concentration,CO_aqi,NO2_concentration,NO2_aqi,O3_concentration,O3_aqi,SO2_concentration,SO2_aqi,PM2.5_concentration,PM2.5_aqi,PM10_concentration,PM10_aqi,overall_aqi,area
0,Washington,293.73,3,13.02,16,62.94,62,4.47,6,9.21,29,11.43,10,62,Washington D.C.
1,Arlington,527.38,5,67.86,84,7.06,5,1.74,2,15.09,48,20.53,19,84,Virginia
2,Alexandria,211.95,2,0.73,0,91.55,141,0.61,0,22.61,64,55.66,50,141,Virginia
3,Fairfax,273.71,3,8.74,10,69.38,82,4.95,7,8.53,27,10.41,9,82,Virginia
4,Rockville,287.06,3,10.80,13,62.94,62,4.65,6,8.77,28,10.77,9,62,Maryland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Oxon Hill,360.49,4,26.39,32,43.63,36,4.47,6,11.41,37,14.03,12,37,Maryland
97,Sykesville,283.72,3,9.25,11,62.94,62,3.93,5,8.44,27,10.50,9,62,Maryland
98,Olney,253.68,2,6.00,7,28.97,24,0.03,0,5.09,16,5.66,5,24,Maryland
99,Georgetown,181.91,2,0.03,0,60.08,53,0.10,0,2.49,8,7.90,7,53,Washington D.C.


## Save Output

In [17]:
# This is the dataframe without the area classification
df.to_csv('dmv_air-quality-data.csv')

In [18]:
# dataframe with the area classification
df1.to_csv('dmv-area_air-quality-data.csv')